<a href="https://colab.research.google.com/github/saikrish02/Integrated-Traffic-Signal-Control-System/blob/main/SignalScheduling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

Mounted at /gdrive


In [2]:
%cd /gdrive/MyDrive

/gdrive/MyDrive


In [3]:
!pip install ultralytics

In [4]:

from ultralytics import YOLO

def getCount(source, count):
  model = YOLO('/gdrive/MyDrive/best.pt')
  results = model(source, imgsz = 640, conf = 0.4, verbose =False)
  for r in results:
    #print(r.boxes)
    #print(r.names)
    for x in r.boxes.cls:
      #print(int(x.item()))
      className = r.names.get(int(x.item()))
      #print(className)
      if className == 'PoliceCar':
        continue
      if count.get(className) is None:
        count[className] = 1
      else:
        count[className] += 1
  model = YOLO('/gdrive/MyDrive/runs/detect/train/weights/best.pt')
  result = model(source, imgsz = 640)
  for r in result:
    #print(r.boxes)
    #print(r.names)
    for x in r.boxes.cls:
      #print(int(x.item()))
      if count.get(r.names.get(int(x.item()))) is None:
        count[r.names.get(int(x.item()))] = 1
      else:
        count[r.names.get(int(x.item()))] += 1
  return


In [5]:
weights = {
    'two-wheeler' : 1,
    'auto' : 3,
    'rickshaw' : 3,
    'car' : 5,
    'pickup' : 8,
    'bus' : 20,
    'truck' : 22,
    'AmbulanceOff': 10,
    'AmbulanceOn' : 100,
    'FireEngineOff' : 22,
    'FireEngineOn' : 100,
    'PoliceCar' : 5
}

In [6]:
def calc_weight(count):
    total = 0
    for i in count.keys():
      total += (weights[i] * count[i])
    return total

In [7]:
if __name__ == '__main__':
    # realTime()
    #images_folder = glob.glob("images/*.jpg")
    # for image in images_folder:
        # from_static_image(image)
    direction = ['North', 'East', 'South', 'West']
    images = ["/content/traffic4.jpg", "/content/traffic5.jpg", "/content/traffic2.jpg", "/content/traffic9.jpg"]
    lane = {}
    for i in range(4):
      count = {}
      print(direction[i])
      getCount(images[i], count)
      weight = calc_weight(count)
      lane[direction[i]] = weight
      print(weight)



North

image 1/1 /content/traffic4.jpg: 384x640 6 autos, 1 bus, 15 cars, 1 pickup, 1 rickshaw, 1 two-wheeler, 53.2ms
Speed: 1.5ms preprocess, 53.2ms inference, 2.4ms postprocess per image at shape (1, 3, 384, 640)
125
East

image 1/1 /content/traffic5.jpg: 384x640 14 autos, 1 bus, 10 cars, 15 two-wheelers, 37.4ms
Speed: 1.7ms preprocess, 37.4ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)
127
South

image 1/1 /content/traffic2.jpg: 640x512 4 autos, 4 buss, 30 cars, 1 pickup, 3 rickshaws, 2 trucks, 15 two-wheelers, 78.0ms
Speed: 4.2ms preprocess, 78.0ms inference, 1.8ms postprocess per image at shape (1, 3, 640, 512)
318
West

image 1/1 /content/traffic9.jpg: 384x640 15 cars, 62.8ms
Speed: 1.3ms preprocess, 62.8ms inference, 1.8ms postprocess per image at shape (1, 3, 384, 640)
75


In [8]:
totalSum = 0
for i in lane.keys():
  totalSum += lane[i]
  print(lane[i])
print(totalSum)

for i in lane.keys():
    lane[i] /= totalSum
lane


125
127
318
75
645


{'North': 0.1937984496124031,
 'East': 0.19689922480620156,
 'South': 0.4930232558139535,
 'West': 0.11627906976744186}

In [9]:
tot_time = 0
for i in lane.keys():
  lane[i] = round(180 * lane[i])
  tot_time += lane[i]
tot_time += 12
lane

{'North': 35, 'East': 35, 'South': 89, 'West': 21}

In [10]:
signal_time = []


for i in range(4):
    print(direction[i].upper())
    x = []
    print(f'Green time : {lane[direction[i]]}sec')
    x.append(lane[direction[i]])
    print('Yellow time : 3sec')
    x.append(3)
    print(f'Red time : {tot_time - lane[direction[i]] - 3}sec')
    x.append(tot_time - lane[direction[i]] - 3)
    print(f'Turn : {i+1}\n')
    signal_time.append(x)
signal_time

NORTH
Green time : 35sec
Yellow time : 3sec
Red time : 154sec
Turn : 1

EAST
Green time : 35sec
Yellow time : 3sec
Red time : 154sec
Turn : 2

SOUTH
Green time : 89sec
Yellow time : 3sec
Red time : 100sec
Turn : 3

WEST
Green time : 21sec
Yellow time : 3sec
Red time : 168sec
Turn : 4



[[35, 3, 154], [35, 3, 154], [89, 3, 100], [21, 3, 168]]

In [12]:
!pip install arcade
!pip install colorama

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.4/39.4 MB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 965.3/965.3 kB 69.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 66.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 996.5/996.5 kB 59.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.7/42.7 kB 6.1 MB/s eta 0:00:00
  Attempting uninstall: pillow
    Found existing installation: Pillow 9.4.0
    Uninstalling Pillow-9.4.0:
      Successfully uninstalled Pillow-9.4.0


In [11]:
import arcade
from colorama import Fore, Style
signal_seq = []

for i in range(4):
    temp = []
    red1 = 0
    red2 = 0
    for j in range(i):
        red1 += signal_time[j][0]
        red1 += 3
    temp.append(red1)
    temp.append(signal_time[i][0])
    temp.append(3)
    for j in range(i+1, 4):
        red2 += signal_time[j][0]
        red2 += 3
    temp.append(red2)
    signal_seq.append(temp)


def print_signal_time(direc, signaltime):
    print(Style.BRIGHT + direc)
    print(Fore.RED + Style.BRIGHT + f'{signaltime[0]} sec')
    print(Fore.GREEN + Style.BRIGHT + f'{signaltime[1]} sec')
    print(Fore.YELLOW + Style.BRIGHT + f'{signaltime[2]} sec')
    print(Fore.RED + Style.BRIGHT + f'{signaltime[3]} sec')
    print(Style.RESET_ALL)

print_signal_time('NORTH', signal_seq[0])
print_signal_time('EAST', signal_seq[1])
print_signal_time('SOUTH', signal_seq[2])
print_signal_time('WEST', signal_seq[3])


NORTH
0 sec
35 sec
3 sec
154 sec

EAST
38 sec
35 sec
3 sec
116 sec

SOUTH
76 sec
89 sec
3 sec
24 sec

WEST
168 sec
21 sec
3 sec
0 sec



In [ ]:
SCREEN_WIDTH = 800
SCREEN_HEIGHT = 600
SCREEN_TITLE = "Traffic Signal"

class MyTimer(arcade.Window):
    def setup(self, sig):
        arcade.set_background_color(arcade.color.WHITE)
        self.i = 0
        self.total_time = sig[0]
        self.signal = sig.copy()

    def on_draw(self):
        arcade.start_render()
        if int(self.total_time) < 0:
            self.i += 1
            if self.i > 3:
                exit(0)
            self.total_time = int(self.signal[self.i])
        seconds = int(self.total_time)
        output = f"{seconds:02d}"
        if self.i == 0 or self.i == 3:
            arcade.draw_text(output, 325, 280, arcade.color.DARK_CANDY_APPLE_RED, 100)
        elif self.i == 1:
            arcade.draw_text(output, 325, 280, arcade.color.BOTTLE_GREEN, 100)
        elif self.i == 2:
            arcade.draw_text(output, 325, 280, arcade.color.DARK_TANGERINE, 100)

    def on_update(self, delta_time):
        self.total_time -= delta_time


window = MyTimer()
window.setup(signal_seq[0])
arcade.run()